In [16]:
### 원본스케일 but 개수 줄여서 테스팅
### 일단 배치 사이즈 늘리는게 최우선이긴함 
### 빠르게 보고하고 새로 학습할때는 자원이 얼마나 필요할지 계산을 좀 해보자
### 배치 사이즈 왜 못늘리는지 다시 확인

In [17]:
import os
list_coords = os.listdir(f'/raid/wlqor98/patch_info/coords/patch{224}_{64000}_level{0}')
#print(len(list_coords))
for path in list_coords:
    pass
    #print(len(os.listdir(f'./patch_info/coords/patch{224}_{64000}_level{0}/{path}') ))

In [18]:
is_multi_gpu = False

save_patch_samples = False
show_val_slides = False

show_accuracy = False
show_slide_accuracy = False
show_gradcam = False
show_slide_pred_visualizer = False
show_filter_analysis = False

use_original_dataset = False # = !new_sampled_dataset(use_coord_dataset) # original : no coord data
# use_npz = False #npz 차이 미미함

if use_original_dataset:
    if show_slide_pred_visualizer:
        print('you cannot use slide pred visualizer')

train = True
learning_rate = 0.001
training_epochs = 3# 이 이상부터 오버피팅 일어남 (7로 했을때)
batch_size = 8 # but we will feed 4 data in getitem 

use_weight_decay = False
weight_decay = 0.001 # 0.0001에서 0.001로 바꿈 (overfitting 너무 심해서)

is_continous_training = False
show_pbar = True

# control dataset
is_year_sort = True
show_year_info = False # total year
show_dataset_year_info = False


use_val_model = True
use_exist_model = False

# control model (exist model)
is_year_sort_model = True
is_l2_norm_model = True

is_cross_model_test = False # 위의 변수로 자동 할당 가능하다. 우선 보류
if show_slide_pred_visualizer == False:
    is_cross_model_test = False # 테스트를 교차검증 식으로 할건지에 대한 변수

if use_exist_model:
    #original sort : data불러오는 폴더의 indexing이 hash number 같은 느낌으로 되어있어서 어쨌든, 어떤 기준을 말하기는 애매함
    # indexing 예시 : 0a2a3ea8-fe81-42e6-b313-e2d08d126dcb : TCGA-BP-4970 # 어쨌든 original sort는 이 순서대로 학습한 것임
    model_name = None
    if is_year_sort_model and is_l2_norm_model:
        model_name = 'l2_year_sort'
    elif is_year_sort_model and not is_l2_norm_model:
        model_name = 'year_sort'
    elif not is_year_sort_model and not is_l2_norm_model:
        model_name = 'original_sort'
    elif not is_year_sort_model and is_l2_norm_model:
        model_name = 'l2_original_sort'

    print(f'model name : {model_name}')


# for original sort
ratio_train=0.8
ratio_val=0.1
ratio_test=0.1

level = 0
patch_size = 224
patches_num = 4000 * 4 * 4
#patches_num = 128
print('test 용으로 128개 씩 하는거 먼저 해보자 얼마나 걸리는지 보게')

# coords와 slide를 통해서 sample img 불러오는 방법 사용
patch_samples_root = f'/raid/wlqor98/patch_info/coords/patch{patch_size}_{patches_num}_level{level}'
patch_coords_root = f'/raid/wlqor98/patch_info/coords/patch{patch_size}_{patches_num}_level{level}'
img_root = f'/raid/wlqor98/Datasets/Guri_hospital/KIRC_TCGA_Dx_images_level2_downsample_whole/'
svs_root = '/raid/Datasets/Guri_hospital/KIRC_TCGA_Dx_images/'

slide_info_root = '/raid/wlqor98/Datasets/Pretrain/guri_test'


import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')    # 경고 무시

import openslide
from multiprocessing import Pool
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn # model!
import torch.nn.functional as F # loss function!
import torch.optim as optim # optimizer!
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import Dataset, DataLoader # 데이터로더
import random
import torch.nn.init
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import copy
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

import re


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("다음 기기로 학습합니다:", device)

random_seed = 1
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if device == 'cuda':
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


class DatasetMaker:
    def __init__(self, is_temp_test=False, level=2, patch_size=64, patches_num=1024, patch_samples_root=None, img_root=None, svs_root=None):
        self.img_root = img_root
        self.xlsx_root = '/raid/Datasets/Guri_hospital/KIRC_TCGA_Dx_Clinical Data/2021-06-30_KIRC_clinical.xlsx'
        self.clinic_data = None
        self.patch_size = patch_size
        self.patch_samples_root = patch_samples_root
        self.svs_root = svs_root
    

    def make_and_load_clinic_data_xlsx(self):
        xlsx_root = self.xlsx_root

        df = pd.read_excel(xlsx_root)
        clinic_data = df[ ['PathologyReportFileName', 'DSS1', 'OverallSurvivalStatus', 'year_of_initial_pathologic_diagnosis'] ]
        clinic_data.rename(columns = {'OverallSurvivalStatus' : 'OSS'}, inplace = True)

        clinic_data.loc[clinic_data['OSS'] =='LIVING', 'OSS'] = 1
        clinic_data.loc[clinic_data['OSS'] =='DECEASED', 'OSS'] = 0
        clinic_data.dropna(inplace = True)
        clinic_data.reset_index(inplace = True)

        # for cutting pathology name length to 12
        length = len(clinic_data['PathologyReportFileName'])
        for i in range(length): clinic_data.loc[clinic_data['PathologyReportFileName'] == clinic_data['PathologyReportFileName'][i], 'PathologyReportFileName'] = clinic_data['PathologyReportFileName'][i][:12]
        clinic_data.head()
        clinic_data.to_excel('./clinic_data.xlsx')
        self.clinic_data = clinic_data

    def get_img_dictionary_and_name_list(self, is_DSS_0_used=False, is_label_ratio_same=False):
        img_root = self.img_root
        clinic_data = self.clinic_data

        directory_candidates = os.listdir(img_root)
        # d is for dictionary
        # l is for list
        d_img = {}

        OSS_cnt = 0
        DSS_cnt = 0
        label_table = np.zeros((2,2))
        for candidate in directory_candidates:
            if len(candidate) == 36:
                img_path = os.path.join(img_root, candidate)
            if len(os.listdir(img_path)) == 0: continue
            img = os.listdir(img_path)[0]
            name = img[:12]

            if name in d_img:
                print(f'dup img name : {name}')
                continue
            
            ## DSS or OSS label checker
            label = clinic_data[clinic_data['PathologyReportFileName'] == name]
            if len(label['OSS']) != 1 or len(label['DSS1']) != 1: continue
            else : [OSS,DSS] = [list(label['OSS'])[0], list(label['DSS1'])[0]]
            

            if not is_DSS_0_used:
                if DSS == 0 and OSS == 0: # 죽었는데 암으로 죽지는 않은 사람
                    continue
            
                
            if OSS == 1:
                OSS_cnt +=1
                if is_label_ratio_same:
                    if OSS_cnt > 100:
                        continue

            if DSS == 1:
                DSS_cnt +=1
                if is_label_ratio_same:
                    if DSS_cnt > 100:
                        continue
            
            label_table[OSS][DSS] += 1
            year_of_initial_pathologic_diagnosis = int(label['year_of_initial_pathologic_diagnosis'])
            
          
            svs = img[:-3] + 'svs'
            d_img[name] = {'img_root': os.path.join(img_path, img), 'OSS' : OSS, 'DSS' : DSS, 
            'svs_root' : os.path.join(self.svs_root, candidate, svs), 'year_of_initial_pathologic_diagnosis' : year_of_initial_pathologic_diagnosis}
        
        for OSS in [0,1]:
            for DSS in [0,1]:
                print(f"OSS : {OSS}, DSS : {DSS} = {label_table[OSS][DSS]}")
        print(label_table)

        l_name = sorted(list(set(d_img.keys())))
        # print(l_name)

        return d_img, l_name
        
    def load_image(self, target_dir, img_name, candidate):
        img = np.load(f'{target_dir}/{img_name}/{candidate}')
        return img
 
        
    def get_x_train_for_image(self, img_name):
        target_dir = f'{self.patch_samples_root}/{img_name}'
        candidates = os.listdir(target_dir)
        print(len(candidates))
        for i, candidate in enumerate(candidates):
            if candidate[-3:] == 'npy':
                img = np.load(f'{target_dir}/{candidate}')
                print(img.shape)
                plt.imshow(img)
                plt.show()
            if i>10:
                break


DM = DatasetMaker(is_temp_test=False, level=level, patch_size=patch_size, patches_num=patches_num, patch_samples_root=patch_samples_root, img_root=img_root, svs_root=svs_root)

DM.make_and_load_clinic_data_xlsx()
d_img, l_name = DM.get_img_dictionary_and_name_list(is_DSS_0_used=False, is_label_ratio_same=False)


test 용으로 128개 씩 하는거 먼저 해보자 얼마나 걸리는지 보게
다음 기기로 학습합니다: cuda
dup img name : TCGA-T7-A92I
dup img name : TCGA-AK-3427
dup img name : TCGA-MM-A564
dup img name : TCGA-A3-3365
dup img name : TCGA-T7-A92I
OSS : 0, DSS : 0 = 0.0
OSS : 0, DSS : 1 = 108.0
OSS : 1, DSS : 0 = 338.0
OSS : 1, DSS : 1 = 0.0
[[  0. 108.]
 [338.   0.]]


In [19]:
patch_list = os.listdir(patch_samples_root)
print(len(patch_list))

len(l_name)

import copy

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def make_target_slide_list(root, d_img, purpose='train', slide_name=None, is_year_sort=False, show_year_info=False, show_dataset_year_info=False):
  print(f'is_year_sort : {is_year_sort}')
  folders = os.listdir(root)

  ######### show year info of data
  if purpose != 'slide' and purpose != 'gradcam':
    year_list = []
    for img_name in folders:
      year = d_img[img_name]['year_of_initial_pathologic_diagnosis'] 
      year_list.append(year)
    
    # Pair each folder with its corresponding year
    folder_year_pairs = zip(folders, year_list)

    # Sort the pairs by year
    sorted_pairs = sorted(folder_year_pairs, key=lambda x: x[1])

    # Extract the sorted folders
    # if we use monthly criteria, we will use sorted_folders
    sorted_folders = [folder for folder, _ in sorted_pairs]

    min_year = min(year_list)
    max_year = max(year_list)
    total_year_bins = range(min_year, max_year)

    if show_year_info:
      plt.hist(year_list, bins=total_year_bins, edgecolor='black')
      plt.xlabel('Year')
      plt.ylabel('Count')
      plt.title(f'Diagnosis Years of Slide (whole dataset)')
      plt.grid(False)
      plt.show()
      plt.clf()

      label_1_year_list = []
      label_0_year_list = []
      for img_name in folders:
        year = d_img[img_name]['year_of_initial_pathologic_diagnosis'] 
        OSS = d_img[img_name]['OSS']
        DSS = d_img[img_name]['DSS']
    
        if OSS == 1 and DSS == 0:
          label_0_year_list.append(year)
        elif DSS == 1 and OSS == 0:
          label_1_year_list.append(year)
        elif DSS == 0 and OSS == 0:
          print('are you using all data?')
        elif DSS == 1 and OSS == 1:
          print('false labeled')

      # integer로 하려면 아래의 코드 참고 but 굳이 필요 없음
      # fig, ax = plt.subplots(figsize=(6,4))
      # ax.locator_params(axis='y', integer=True)
      plt.hist(label_1_year_list, bins=total_year_bins, edgecolor='black')
      plt.xlabel('Year')
      plt.ylabel('Count')
      plt.title(f'Diagnosis Years of Slide label_1 (whole dataset)')
      plt.grid(False)
      plt.show()
      plt.clf()

      plt.hist(label_0_year_list, bins=total_year_bins, edgecolor='black')
      plt.xlabel('Year')
      plt.ylabel('Count')
      plt.title(f'Diagnosis Year of Slide label_0 (whole dataset)')
      plt.grid(False)
      plt.show()
      plt.clf()
    ##################################


  if purpose == 'slide' or purpose == 'gradcam':
    folders = [slide_name]
  elif is_year_sort:
    year_count = 0
    current_year = 0
    test_year = 2009
    train_folders = []
    val_folders = []
    test_folders = []
    for i, (folder, year) in enumerate(sorted_pairs):
      if i == len(sorted_pairs) - 1:
        if show_year_info:
            pass
            #print(f'year {current_year}, year_count {year_count}')
      if current_year != year:
        if current_year != 0:
          if show_year_info:
            pass
            #print(f'year {current_year}, year_count {year_count}')
        current_year = year
        year_count = 0
      year_count += 1
      if year in [2002, 2011, 2012, 2013]:
        test_folders.append(folder)
      elif year in [2008, 2009]:
        val_folders.append(folder)
      else:
        train_folders.append(folder)

    if purpose == 'train':
      folders = train_folders
    elif purpose == 'val':
      folders = val_folders
    elif purpose == 'test':
      folders = test_folders
  else:
    if purpose == 'train':
      folders = folders[:int(len(folders)*ratio_train)]
    elif purpose == 'val':
      folders = folders[int(len(folders)*ratio_train):int(len(folders)*(ratio_train + ratio_val))]
    elif purpose == 'test':
      folders = folders[int(len(folders)*(ratio_train + ratio_val)):]
    else:
      print('wrong purpose')

  if show_dataset_year_info and purpose != 'slide' and purpose != 'gradcam':
    print('show_dataset_year_info!!')
    year_list = []
    for img_name in folders:
      year = d_img[img_name]['year_of_initial_pathologic_diagnosis'] 
      year_list.append(year)
    
    # Pair each folder with its corresponding year
    folder_year_pairs = zip(folders, year_list)

    # Sort the pairs by year
    sorted_pairs = sorted(folder_year_pairs, key=lambda x: x[1])

    # Extract the sorted folders
    # if we use monthly criteria, we will use sorted_folders
    sorted_folders = [folder for folder, _ in sorted_pairs]


    plt.hist(year_list, bins=total_year_bins, edgecolor='black')
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.title(f'Diagnosis Years of Slide ({purpose} dataset)')
    plt.grid(False)
    plt.show()
    plt.clf()

    label_1_year_list = []
    label_0_year_list = []
    for img_name in folders:
      year = d_img[img_name]['year_of_initial_pathologic_diagnosis'] 
      OSS = d_img[img_name]['OSS']
      DSS = d_img[img_name]['DSS']
  
      if OSS == 1 and DSS == 0:
        label_0_year_list.append(year)
      elif DSS == 1 and OSS == 0:
        label_1_year_list.append(year)
      elif DSS == 0 and OSS == 0:
        print('are you using all data?')
      elif DSS == 1 and OSS == 1:
        print('false labeled')

    plt.hist(label_1_year_list, bins=total_year_bins, edgecolor='black')
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.title(f'Diagnosis Years of Slide label_1 ({purpose} dataset)')
    plt.grid(False)
    plt.show()
    plt.clf()

    plt.hist(label_0_year_list, bins=total_year_bins, edgecolor='black')
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.title(f'Diagnosis Year of Slide label_0 ({purpose} dataset)')
    plt.grid(False)
    plt.show()
    plt.clf()
  
    
  return folders


446


In [20]:

import torch.nn.init
import torch


from torch.optim import AdamW
from torch.utils.data import DataLoader, DistributedSampler

# 1. initialize process group
if is_multi_gpu:
  os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
  os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"  # Set the GPUs 2 and 3 to use
  import torch.distributed as dist
  from torch.nn.parallel import DistributedDataParallel
  dist.init_process_group("nccl")
  rank = dist.get_rank()
  torch.cuda.set_device(rank)
  device = torch.cuda.current_device()
  world_size = dist.get_world_size()





class CustomDataset(Dataset):
  def __init__(self, root, d_img, purpose='train', is_pixel_value_0_to_1=True, slide_name=None, show_detail=True, is_year_sort=False, show_year_info=False, show_dataset_year_info=False, patches_num=patches_num):

      self.root = root
      self.regions_root = f'{root}/cropped_regions'
      self.coords_root = f'{root}/patch_coords'
      self.origins_root = f'{root}/region_globals_origins' # 얘는 어차피 일단 학습에는 안쓰임! => 나중에 얘까지 쓰는 버전으로 작게 테스트 해본 뒤에 전체 학습 시작해야함

      self.whole_coords_data_path = f'{root}/preprocessed_data/whole_coords_{purpose}{is_year_sort}{ratio_train}.npy'
      self.whole_labels_data_path = f'{root}/preprocessed_data/whole_labels_{purpose}{is_year_sort}{ratio_train}.npy'
      self.whole_region_names_data_path = f'{root}/preprocessed_data/whole_region_names_{purpose}{is_year_sort}{ratio_train}.npy'

      self.file_list = [] # patch file
      self.target_slide_list = []
      self.labels = None
      self.d_img = d_img
      self.purpose = purpose
      self.is_pixel_value_0_to_1 = is_pixel_value_0_to_1
      self.label_table = np.zeros((2,2))
      self.show_detail = show_detail
      self.patches_num = patches_num
      self.svs_openslide_list = []
      
      self.whole_coords_data = None

      self.region_file_name_data = None

      #여기 patch_samples_root 안쓰게 통일해줘야함
      self.target_slide_list = make_target_slide_list(patch_samples_root, d_img, purpose=purpose, slide_name=slide_name, is_year_sort=is_year_sort ,show_year_info=show_year_info, show_dataset_year_info=show_dataset_year_info)
      if self.show_detail:
        print(self.target_slide_list)
        print(f'img cnt : {len(self.target_slide_list)}')


      if os.path.exists(self.whole_coords_data_path):
        self.whole_coords_data = np.load(self.whole_coords_data_path)
        self.region_file_name_data = np.load(self.whole_region_names_data_path)
        self.labels = np.load(self.whole_labels_data_path)
      else:
        cnt_regions = self.get_valid_regions_count()
        print(f'cnt_regions:{cnt_regions}')
        whole_coords_data = np.zeros((cnt_regions,32,2), dtype=np.uint8)
        self.region_file_name_data = np.zeros((cnt_regions), dtype='a20') # 20개의 character를 저장가능
        self.labels = np.zeros((cnt_regions))
        idx_regions = 0

        for img_name in self.target_slide_list:
          OSS = d_img[img_name]['OSS']
          DSS = d_img[img_name]['DSS']
          svs_root = d_img[img_name]['svs_root']
          slide = openslide.OpenSlide(svs_root)
          self.svs_openslide_list.append(slide)
          
          if OSS == 1 and DSS == 0:
            label = np.zeros((1))
          elif DSS == 1 and OSS == 0:
            label = np.ones((1))
          elif DSS == 0 and OSS == 0:
            print('are you using all data?')
          elif DSS == 1 and OSS == 1:
            print('false labeled')
            
          self.label_table[OSS][DSS] += 1
          # self.labels.append(label)

          coords_path = f'{self.coords_root}/{img_name}'
          # print(os.listdir(img_root)[:3]) 패치들이 순서대로 list에 삽입되지는 않는다.
          # print(len(os.listdir(img_root)))
          
          coords_file_list = os.listdir(coords_path)
          for coords_file_name in coords_file_list:
            coords_file_path = f'{coords_path}/{coords_file_name}'
            coords = np.load(coords_file_path)
            
            
            if coords.shape[0] == 32:
              whole_coords_data[idx_regions] = coords.astype(int)
              region_file_name = f'{coords_file_name[:-11]}' # 11개를 잘라먹겠다는 뜻임
              self.region_file_name_data[idx_regions] = region_file_name
              self.labels[idx_regions] = label
              idx_regions += 1

              # print(f'{self.regions_root}/{img_name}/{coords_file_name[:-11]}.npy')
      
        self.whole_coords_data = whole_coords_data
        np.save(self.whole_coords_data_path, self.whole_coords_data)
        np.save(self.whole_region_names_data_path, self.region_file_name_data)
        np.save(self.whole_labels_data_path, np.array(self.labels))


      print(f'{self.whole_coords_data.shape}/{self.region_file_name_data.shape}')
      self.labels = torch.FloatTensor(self.labels).unsqueeze(1)  

      for OSS in [0,1]:
        for DSS in [0,1]:
          if self.show_detail:
            print(f"OSS : {OSS}, DSS : {DSS} = {self.label_table[OSS][DSS]}")
      if self.show_detail:
        print(self.label_table)

  def get_target_slide_list(self):
    return self.target_slide_list

  def get_valid_regions_count(self):
    if self.purpose == 'train':
      return 1073464 # 이거 train, test, val 다르기 때문에 그냥 직접계산하는게 맞다
    elif self.purpose == 'val':
      return 131741
    elif self.purpose == 'test':
      return 127603

    cnt_regions = 0
    for img_name in self.target_slide_list:
      coords_path = f'{self.coords_root}/{img_name}'    
      coords_file_list = os.listdir(coords_path)
      for coords_file_name in coords_file_list:
          coords_file_path = f'{coords_path}/{coords_file_name}'
          coords = np.load(coords_file_path)
          if cnt_regions == 0:
            print(coords.dtype)
          if coords.shape[0] == 32:
            cnt_regions += 1

    print(cnt_regions)
    return cnt_regions
  
  def numpy_to_tensor(self, x_data):
    x_data = torch.FloatTensor(x_data)
    x_data = x_data.permute(2, 0, 1)  # Permute the dimensions
    return x_data

  def __len__(self):
    # coord_list는 svs당 하나
    return (len(self.region_file_name_data) * 32) // 4 # Since we are getting two items at a time


  def __getitem__(self, idx):
    idx = 4*idx
    idx_2 = idx + 1
    idx_3 = idx + 2
    idx_4 = idx + 3
    # numpy로 사진 저장하기 (/255는 numpy uint 형식으로 저장 )
    # 한장 슬라이드에서 나중에 
    # 32 : coords per region
    # 이렇게 똑같이 하는데, region에서 뽑은거랑 svs에서 뽑는거랑 비교해보기
    region_num = idx // 32
    region_file_name = self.region_file_name_data[region_num]
    region_file_name = region_file_name.decode('utf-8')
    region = np.load(f'{self.regions_root}/{region_file_name[:12]}/{region_file_name}.npy')


    coord = self.whole_coords_data[region_num][idx % 32]
    x = coord[0]
    y = coord[1]
    patch = region[y:y+224, x:x+224]
    x_data = self.numpy_to_tensor(patch)
    x_data = x_data / 255
    label = self.labels[region_num]

    coord = self.whole_coords_data[region_num][idx_2 % 32]
    x = coord[0]
    y = coord[1]
    patch = region[y:y+224, x:x+224]
    x_data_2 = self.numpy_to_tensor(patch)
    x_data_2 = x_data / 255
    label_2 = self.labels[region_num]

    coord = self.whole_coords_data[region_num][idx_3 % 32]
    x = coord[0]
    y = coord[1]
    patch = region[y:y+224, x:x+224]
    x_data_3 = self.numpy_to_tensor(patch)
    x_data_3 = x_data / 255
    label_3 = self.labels[region_num]

    coord = self.whole_coords_data[region_num][idx_4 % 32]
    x = coord[0]
    y = coord[1]
    patch = region[y:y+224, x:x+224]
    x_data_4 = self.numpy_to_tensor(patch)
    x_data_4 = x_data / 255
    label_4 = self.labels[region_num]

    return (x_data, label), (x_data_2, label_2), (x_data_3, label_3), (x_data_4, label_4)


class ModelTrain:
  def __init__(self, PATH, PATH_full_epochs, model, dataset, val_dataset, test_dataset, criterion, optimizer, training_epochs=30, batch_size=32, shuffle=True, learning_rate=0.001):
    self.model = model
    self.dataset = dataset
    self.criterion = criterion
    self.optimizer = optimizer
    self.training_epochs = training_epochs
    self.batch_size = batch_size
    self.num_workers = 2

    if is_multi_gpu:
      train_sampler = DistributedSampler(
        dataset,
        num_replicas=world_size,
        rank=rank,
        shuffle=True,
      )
      self.data_loader = DataLoader(dataset=dataset, sampler=train_sampler, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=self.num_workers,pin_memory=True) # 미니배치사이즈를 말한다
    else:
      self.data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True, num_workers=self.num_workers,pin_memory=True) # 미니배치사이즈를 말한다
    self.val_data_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True, num_workers=self.num_workers, pin_memory=True) # 미니배치사이즈를 말한다
    self.test_data_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True, num_workers=self.num_workers, pin_memory=True) # 미니배치사이즈를 말한다
    self.PATH = PATH + '_use_val.pt'
    self.PATH_full_epochs = PATH + '_full_epochs.pt'
    self.PATH_first = PATH + '_first.pt'
    self.scheduler = StepLR(self.optimizer, step_size=1, gamma=0.1)

    print(f'lr : {self.optimizer.param_groups[0]["lr"]}')


  def train_model(self, use_val=False):
    costs = []
    val_loss_list = []
    test_loss_list = []
    min_val_loss = 30000
    min_val_loss_epoch = 1
    best_model = None
    best_model_optimizer = None

    total_batch = len(self.data_loader)
    val_total_batch = len(self.val_data_loader)
    test_total_batch = len(self.test_data_loader)
    print('총 배치의 수 : {}'.format(total_batch))
    for epoch in range(self.training_epochs):
      self.model.train()
      # if epoch == 1 or epoch == 3:
      #   self.scheduler.step()
      avg_cost = 0
      if show_pbar:
        pbar = tqdm(total = total_batch)
      for (X1, Y1), (X2, Y2), (X3, Y3), (X4, Y4) in self.data_loader:
        
        # Combine the two batches
        X = torch.cat([X1, X2, X3, X4], dim=0).to(device)
        Y = torch.cat([Y1, Y2, Y3, Y4], dim=0).to(device)

        hypothesis = self.model(X)
        cost = self.criterion(hypothesis, Y)

        self.optimizer.zero_grad()
        cost.backward()
        self.optimizer.step()

        avg_cost += cost / total_batch
        if show_pbar:
          pbar.update(1)

      costs.append(avg_cost.cpu().detach().numpy())
      
      if use_val == True:
        self.model.eval()
        val_loss = 0
        test_loss = 0
        with torch.no_grad():
          for (X1, Y1), (X2, Y2), (X3, Y3), (X4, Y4) in self.val_data_loader:
            # Combine the two batches
            X = torch.cat([X1, X2, X3, X4], dim=0).to(device)
            Y = torch.cat([Y1, Y2, Y3, Y4], dim=0).to(device)
            hypothesis = self.model(X)
            loss = self.criterion(hypothesis, Y)
            val_loss += loss / val_total_batch
          
          for (X1, Y1), (X2, Y2), (X3, Y3), (X4, Y4) in self.test_data_loader:
            # Combine the two batches
            X = torch.cat([X1, X2, X3, X4], dim=0).to(device)
            Y = torch.cat([Y1, Y2, Y3, Y4], dim=0).to(device)
            hypothesis = self.model(X)
            loss = self.criterion(hypothesis, Y)
            test_loss += loss / test_total_batch

        val_loss_list.append(val_loss.cpu().detach().numpy())
        test_loss_list.append(test_loss.cpu().detach().numpy())

        
        if val_loss < min_val_loss:
          min_val_loss = val_loss
          min_val_loss_epoch = epoch+1
          best_model = self.model.state_dict()
          best_model_optimizer = self.optimizer.state_dict()
          # 추후에 deepcopy로 바꾸기. 위의 것 자체가 object pointer를 반환하는 듯 하다...
          torch.save({
            'epoch': min_val_loss_epoch,
            'model_state_dict': best_model,
            'optimizer_state_dict': best_model_optimizer,
            'loss': min_val_loss,
          }, self.PATH)

        
          # if self.optimizer.param_groups[0]["lr"] < 1e-5: self.optimizer.param_groups[0]["lr"] = 1e-5


        # elif epoch > 0:
        #   self.optimizer.param_groups[0]["lr"] *= 0.1
        #   print(f'epoch+1 : {epoch+1}, learning_rate : {self.optimizer.param_groups[0]["lr"]}')
        #   if self.optimizer.param_groups[0]["lr"] < 1e-5: self.optimizer.param_groups[0]["lr"] = 1e-5



      print('[Epoch: {:>4}] cost = {:>.5}  val_loss = {:>.5} test_loss = {:>.5}'.format(epoch + 1, avg_cost, val_loss, test_loss))
      print(f'Epoch : {epoch+1}, learning_rate : {self.optimizer.param_groups[0]["lr"]}')
      if epoch+1 == 2: # epoch 3부터 10배 작아진 learning rate 적용
        self.optimizer.param_groups[0]["lr"] *= 0.1
      

    print(f'lowest loss on validation set : {min_val_loss}')


    torch.save({
      'epoch': self.training_epochs,
      'model_state_dict': self.model.state_dict(),
      'optimizer_state_dict': self.optimizer.state_dict(),
      'loss': costs[-1],
    }, self.PATH_full_epochs)

    x = range(1, len(costs)+1)

    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.plot(x, costs, 'b')
    plt.plot(x, val_loss_list, 'r')
    plt.plot(x, test_loss_list, 'g')
    plt.grid(True)
    plt.xticks(x)
    plt.savefig('savefig_loss_graph.png')
    plt.show()
    plt.clf()

class LoadPretrainedModel:
  def __init__(self):
    pass

  def load_model_ResNet_modified(self, version='resnet18'):
    # Load pre-trained ResNet18 model
    model = models.resnet18(pretrained=True)
    # Modify the first layer of the model to accept 3x64x64 input
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 1),
        nn.Sigmoid()
    )
    return model

model = LoadPretrainedModel().load_model_ResNet_modified().to(device)

if is_continous_training:
  checkpoint = torch.load('epochs_10_lr=0.001_04-12-18:20_use_val.pt')
  model.load_state_dict(checkpoint['model_state_dict'])

criterion = torch.nn.BCELoss().to(device)
intend_over_fitting = False

if intend_over_fitting:
    learning_rate = 0.0003 # linearization 가정을 만족할 만큼 작은 스텝사이즈여야!
    training_epochs = 15 # 이 이상부터 오버피팅 일어남 (7로 했을때)



import time
timestamp = time.strftime('%m-%d-%H:%M', time.localtime(time.time()))

PATH = f"epochs_{training_epochs}_lr={learning_rate}_{timestamp}"
PATH_full_epochs = f"epochs_{training_epochs}_lr={learning_rate}_{timestamp}"

print(f'PATH_full_epochs : {PATH_full_epochs}_full_epochs')

if is_cross_model_test:
  pass
  # 그냥 직접 따져주는게 더 정확 (model이랑 sort랑 반대로 해주면 cross model test다)
  # is_cross_model_year_sort = not is_year_sort # is_year_sort
  cross_dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='train', is_year_sort=is_year_sort, show_year_info=show_year_info, show_dataset_year_info=show_dataset_year_info)
  cross_val_dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='val', is_year_sort=is_year_sort, show_year_info=False, show_dataset_year_info=show_dataset_year_info)
  cross_test_dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='test', is_year_sort=is_year_sort, show_year_info=False, show_dataset_year_info=show_dataset_year_info)
  cross_slide_list = cross_dataset.get_target_slide_list()
  cross_val_slide_list = cross_val_dataset.get_target_slide_list()
  cross_test_slide_list = cross_test_dataset.get_target_slide_list()

  print(cross_slide_list)
  print(cross_val_slide_list)
  print(cross_test_slide_list)
dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='train', is_year_sort=is_year_sort, show_year_info=show_year_info, show_dataset_year_info=show_dataset_year_info)
val_dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='val', is_year_sort=is_year_sort, show_year_info=False, show_dataset_year_info=show_dataset_year_info)
test_dataset = CustomDataset(root = slide_info_root, d_img=d_img, purpose='test', is_year_sort=is_year_sort, show_year_info=False, show_dataset_year_info=show_dataset_year_info)


  



print(f'PATH_val : {PATH}_use_val')
if is_multi_gpu:
  model = DistributedDataParallel(model, device_ids=[device], output_device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

if use_weight_decay:
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

MTR = ModelTrain(PATH=PATH, PATH_full_epochs=PATH_full_epochs, model=model, dataset=dataset, val_dataset=val_dataset, test_dataset=test_dataset, criterion=criterion, optimizer=optimizer, training_epochs=training_epochs, batch_size=batch_size, shuffle=True, learning_rate=learning_rate)



if train:

  MTR.train_model(use_val=True)

if train and use_val_model:
    model = LoadPretrainedModel().load_model_ResNet_modified().to(device)
    checkpoint = torch.load(f'{PATH}_use_val.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
if use_exist_model:
    if is_year_sort_model and is_l2_norm_model:
      PATH = 'epochs_7_lr=0.001_05-13-14:46_use_val' # l2norm 0.001 year sort
      PATH = 'epochs_3_lr=0.001_06-25-16:45_use_val'
    elif is_year_sort_model and not is_l2_norm_model:
      #PATH = 'epochs_8_lr=0.001_05-08-21:20_use_val' # year sort
      PATH = 'epochs_3_lr=0.001_06-18-15:11_use_val'
    elif not is_year_sort_model and is_l2_norm_model:
      PATH = 'epochs_7_lr=0.001_05-11-19:49_use_val' # l2norm 0.001 original sort
    elif not is_year_sort_model and not is_l2_norm_model:
      PATH = 'epochs_10_lr=0.001_04-12-18:20_use_val' # original sort
      # [Epoch: 2] cost = 0.1942  val_loss = 0.5083 test_loss = 0.7353
    #PATH = 'epochs_8_lr=0.001_05-06-22:34_use_val' 

    model = LoadPretrainedModel().load_model_ResNet_modified().to(device)
    checkpoint = torch.load(f'{PATH}.pt')
    model.load_state_dict(checkpoint['model_state_dict'])

print(f'current using model : {PATH}')

# 10분걸림

PATH_full_epochs : epochs_3_lr=0.001_08-22-09:39_full_epochs
is_year_sort : True
['TCGA-B0-5100', 'TCGA-B0-5102', 'TCGA-AK-3425', 'TCGA-BP-4158', 'TCGA-BP-4325', 'TCGA-B0-4810', 'TCGA-B0-4696', 'TCGA-B0-4814', 'TCGA-B0-4811', 'TCGA-B0-4815', 'TCGA-BP-4341', 'TCGA-BP-4169', 'TCGA-B0-4697', 'TCGA-CJ-5681', 'TCGA-BP-4335', 'TCGA-CJ-5679', 'TCGA-BP-4334', 'TCGA-CJ-6031', 'TCGA-B0-5699', 'TCGA-B0-4836', 'TCGA-B0-5709', 'TCGA-BP-4167', 'TCGA-BP-4756', 'TCGA-B0-4845', 'TCGA-B0-4833', 'TCGA-CJ-5678', 'TCGA-BP-4342', 'TCGA-CJ-6028', 'TCGA-B0-5693', 'TCGA-BP-4338', 'TCGA-B0-5812', 'TCGA-CJ-5680', 'TCGA-B0-4828', 'TCGA-BP-4166', 'TCGA-BP-4343', 'TCGA-BP-4959', 'TCGA-B0-4837', 'TCGA-B0-5706', 'TCGA-BP-4766', 'TCGA-BP-5180', 'TCGA-B0-5707', 'TCGA-BP-5174', 'TCGA-CJ-4874', 'TCGA-B0-4843', 'TCGA-CJ-6032', 'TCGA-CJ-4871', 'TCGA-CJ-4638', 'TCGA-CJ-4636', 'TCGA-BP-4960', 'TCGA-CJ-4640', 'TCGA-B0-4849', 'TCGA-B0-4852', 'TCGA-BP-4761', 'TCGA-BP-4760', 'TCGA-CJ-4634', 'TCGA-AK-3428', 'TCGA-CJ-4639', 'TCGA-

  0%|          | 0/1073464 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:

class ModelTest:
  def __init__(self, model):
    self.model = model
  
  def test_on_target_set(self, dataset, test_name='accuracy', return_accuracy=False, show_pbar=show_pbar, show_visualizer=False):
    # Set the model to evaluation mode
    model = self.model
    model.eval()

    num_workers = 2
    if show_visualizer:
      num_workers = 1
    # 이거 map이용해서 multi로 돌리기!
    data_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=num_workers, pin_memory=True)
    
    print(len(dataset))
    data_len = len(dataset)
    batch_len = len(data_loader)
    whole_y_pred_prob = np.zeros((data_len,1))
    whole_y_pred_binary = np.zeros((data_len,1))
    whole_y_target = np.zeros((data_len,1))
    print(whole_y_pred_prob.shape)

    i = 0
    total_correct = 0
    if show_pbar:
      pbar = tqdm(total = batch_len)
    for X, Y in data_loader:
      X = X.to(device)
      Y = Y.to(device)

      hypothesis = model(X)

      with torch.no_grad():
          y_pred_prob = model(X)

      # Convert the predicted probabilities to binary predictions
      y_pred_binary = torch.round(y_pred_prob)
      
      if Y.shape[0] == 32:
        whole_y_pred_prob[i*32:(i+1)*32] = y_pred_prob.cpu().detach().numpy()
        whole_y_pred_binary[i*32:(i+1)*32] = y_pred_binary.cpu().detach().numpy()
        whole_y_target[i*32:(i+1)*32] = Y.cpu().detach().numpy()
      else: 
        whole_y_pred_prob[i*32:i*32 + Y.shape[0]] = y_pred_prob.cpu().detach().numpy()
        whole_y_pred_binary[i*32:i*32 + Y.shape[0]] = y_pred_binary.cpu().detach().numpy()
        whole_y_target[i*32:i*32 + Y.shape[0]] = Y.cpu().detach().numpy()

      i += 1
      # Calculate the accuracy
      correct = (y_pred_binary == Y).sum().item()
      total_correct += correct

      if show_pbar:
        pbar.update(1)
    
    accuracy = total_correct / data_len

    print("Accuracy: {:.2f}%".format(accuracy * 100))

    if return_accuracy:
      return whole_y_target, whole_y_pred_binary, whole_y_pred_prob, accuracy

    return whole_y_target, whole_y_pred_binary, whole_y_pred_prob
     
model.eval()

from sklearn.metrics import precision_recall_curve, auc as auc_ftn
import matplotlib.pyplot as plt

MTE = ModelTest(model=model)

if show_accuracy:
    with torch.no_grad():
        y_val, y_val_pred_binary, y_val_pred_prob = MTE.test_on_target_set(val_dataset)

    fpr, tpr, _ = metrics.roc_curve(y_val, y_val_pred_prob)
    auc = metrics.roc_auc_score(y_val, y_val_pred_prob)

    #create ROC curve
    plt.plot(fpr,tpr,label="AUC="+str(auc))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.grid(True)
    plt.savefig('savefig_val_auc.png')
    plt.show()
    plt.clf()

    # PR_curve
    precision, recall, thresholds = precision_recall_curve(y_val,  y_val_pred_prob)
    area = auc_ftn(recall, precision)

    plt.plot(recall, precision, label=f"PR curve (AUC = {area:.2f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid(True)
    plt.savefig('savefig_val_pr.png')
    plt.show()
    plt.clf()

    with torch.no_grad():
        y_test, y_test_pred_binary, y_test_pred_prob = MTE.test_on_target_set(test_dataset)


    fpr, tpr, _ = metrics.roc_curve(y_test, y_test_pred_prob)
    auc = metrics.roc_auc_score(y_test, y_test_pred_prob)

    #create ROC curve
    plt.plot(fpr,tpr,label="AUC="+str(auc))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.grid(True)
    plt.savefig('savefig_test_auc.png')
    plt.show()
    plt.clf()

    # PR_curve
    precision, recall, thresholds = precision_recall_curve(y_test,  y_test_pred_prob)
    area = auc_ftn(recall, precision)

    plt.plot(recall, precision, label=f"PR curve (AUC = {area:.2f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid(True)
    plt.savefig('savefig_test_pr.png')
    plt.show()
    plt.clf()
    

    with torch.no_grad():
        y_train, y_train_pred_binary, y_train_pred_prob = MTE.test_on_target_set(dataset)


    fpr, tpr, _ = metrics.roc_curve(y_train[:],  y_train_pred_prob[:])
    auc = metrics.roc_auc_score(y_train[:], y_train_pred_prob[:])

    #create ROC curve
    plt.plot(fpr,tpr,label="AUC="+str(auc))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc=4)
    plt.grid(True)
    plt.savefig('savefig_train_auc.png')
    plt.show()
    plt.clf()

    # PR_curve
    precision, recall, thresholds = precision_recall_curve(y_train[:],  y_train_pred_prob[:])
    area = auc_ftn(recall, precision)

    plt.plot(recall, precision, label=f"PR curve (AUC = {area:.2f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid(True)
    plt.savefig('savefig_train_pr.png')
    plt.show()
    plt.clf()


    

def slide_accuracy_test_on_target_set(target_slide_list, d_img, use_threshold=False, threshold=None, show_visualizer=False, show_label_accuracy=True, show_label_only=None, is_cross_model_test=False, dataset_name=None):
  
  cnt_correct = 0
  cnt_wrong = 0
  label_1_cnt = 0
  label_1_correct_cnt = 0
  label_0_cnt = 0
  label_0_correct_cnt = 0

  if use_threshold:
    t_label_1_correct_cnt = 0
    t_label_0_correct_cnt = 0
    t_cnt_correct = 0
    t_cnt_wrong = 0

  model_info_slide_dataset = None

  if not is_cross_model_test:
    if dataset_name == 'val':
      model_info_slide_dataset = 'unseen(val)'
    elif dataset_name == 'test':
      model_info_slide_dataset = 'unseen(test)'
    elif dataset_name == 'train':
      model_info_slide_dataset = 'seen(train)'
      
  for slide_name in target_slide_list:
    if is_cross_model_test:
      is_unseen_slide = False
      for cross_val_slide in cross_val_slide_list:
        if slide_name == cross_val_slide:
          print('this slide is in cross val slide list')
          is_unseen_slide = True
          model_info_slide_dataset = 'unseen(val)'
          break
      for cross_test_slide in cross_test_slide_list:
        if slide_name == cross_test_slide:
          print('this slide is in cross test slide list')
          is_unseen_slide = True
          model_info_slide_dataset = 'unseen(test)'
          break
      if not is_unseen_slide:
        print('this slide is in cross train slide list')
        model_info_slide_dataset = 'seen(train)'



    slide_dataset = CustomDataset(root = patch_samples_root, d_img=d_img, purpose='slide', slide_name=slide_name, show_detail=False)
    
    y_target, y_target_pred_binary_each, y_target_pred_prob_each, img_accuracy = MTE.test_on_target_set(slide_dataset, return_accuracy=True, show_pbar=False, show_visualizer=show_visualizer)
    label = y_target[0]
    
    print(f'slide_name : {slide_name} / label : {label}')

    cnt_1 = 0
    cnt_0 = 0

    for pred in y_target_pred_binary_each:
        if pred == 1:
            cnt_1 += 1
        else:
            cnt_0 += 1
    
    if cnt_1 > cnt_0:
        voted_pred = 1
    else:
        voted_pred = 0

    if use_threshold:
      if cnt_1 / (cnt_1 + cnt_0) > threshold:
        t_voted_pred = 1
      else:
        t_voted_pred = 0
    
    if label == 1:
      label_1_cnt += 1
    else:
      label_0_cnt += 1

    if label == voted_pred:
        print('correct')
        cnt_correct += 1
        if label == 1:
          label_1_correct_cnt += 1
        else:
          label_0_correct_cnt += 1
    else:
        print('wrong')
        cnt_wrong += 1

    if use_threshold:
      if label == t_voted_pred:
        print(f'correct (threshold for label_1 : {threshold*100}%)')
        t_cnt_correct += 1
        if label == 1:
          t_label_1_correct_cnt += 1
        else:
          t_label_0_correct_cnt += 1
      else:
          print(f'wrong (threshold for label_1 : {threshold*100}%)')
          t_cnt_wrong += 1

    if show_visualizer:
      if show_label_only != None:
        if label != show_label_only:
          continue

      img_info = d_img[slide_name]
      img_root = img_info['img_root']
      img = cv2.imread(img_root)
      ###########
      fig, ax = plt.subplots(figsize=(10, 10))
      plt.title(f'{slide_name} / label={int(label[0])}')
      plt.imshow(img[..., ::-1])
      ###########

      #continue

      fig, ax = plt.subplots(figsize=(10, 10))
      plt.title(slide_name + f': {model_info_slide_dataset}' + ' / Accuracy: {:.2f}% / '.format(img_accuracy * 100) + "model: " + model_name)
      ax.imshow(img[..., ::-1])

      target_dir = f'{patch_coords_root}/{slide_name}' 
      candidates = os.listdir(target_dir)
      target_file = candidates[0]
      if not target_file[-3:] == 'npy':
          print('file format error')
      
      coords = np.load(f'{target_dir}/{target_file}')
      color_list = ['g', 'b']
      for i, coord in enumerate(coords):
          x = coord[0]
          y = coord[1]
          
          pred_label = int(y_target_pred_binary_each[i])
          color = color_list[pred_label]
          
          pred_prob = float(y_target_pred_prob_each[i])
          alpha = pred_prob
          if pred_label == 0:
            alpha = 1 - alpha
          
          # pred binary : color # green : label_0 / blue : label_1
          # pred prob : alpha
          ax.add_patch(plt.Rectangle((x, y), patch_size//16, patch_size//16, edgecolor=color, facecolor=color, alpha=alpha))
      plt.show()

  print(f'cnt_correct : {cnt_correct}')
  print(f'cnt_wrong : {cnt_wrong}')
  accuracy = cnt_correct/(cnt_correct + cnt_wrong)
  print("Slide Accuracy: {:.2f}%".format(accuracy * 100))  # dataset의 slide 전체 종합 accuracy를 말하는 것!

  if show_label_accuracy:
    print("label_1 에 대한 정답률: {:.2f}%".format(label_1_correct_cnt/label_1_cnt * 100))
    print("label_0 에 대한 정답률: {:.2f}%".format(label_0_correct_cnt/label_0_cnt * 100))

  if use_threshold:
    print(f'아래는 threshold {threshold*100}% 이상의 패치가 label 1일때 slide label을 1로 지정했을때의 결과')
    print(f'cnt_correct : {t_cnt_correct}')
    print(f'cnt_wrong : {t_cnt_wrong}')
    accuracy = t_cnt_correct/(t_cnt_correct + t_cnt_wrong)
    print("Slide Accuracy: {:.2f}%".format(accuracy * 100))  

    if show_label_accuracy:
      print("label_1 에 대한 정답률: {:.2f}%".format(t_label_1_correct_cnt/label_1_cnt * 100))
      print("label_0 에 대한 정답률: {:.2f}%".format(t_label_0_correct_cnt/label_0_cnt * 100))

img_cnt = len(os.listdir(patch_samples_root))

val_slide_list = val_dataset.get_target_slide_list()
test_slide_list = test_dataset.get_target_slide_list()
train_slide_list = dataset.get_target_slide_list()

if show_slide_accuracy:
    
    slide_accuracy_test_on_target_set(val_slide_list, d_img, use_threshold=True, threshold=0.2)
    slide_accuracy_test_on_target_set(test_slide_list, d_img, use_threshold=True, threshold=0.2)
    slide_accuracy_test_on_target_set(train_slide_list, d_img, use_threshold=True, threshold=0.2)

In [ ]:



if show_slide_pred_visualizer:
  
  dataset_name = 'val'

  if dataset_name == 'val':
    target_slide_list = val_slide_list
  elif dataset_name == 'test':
    target_slide_list = test_slide_list
  elif dataset_name == 'train':
    target_slide_list = train_slide_list
 
  #target_slide_list = ['TCGA-B0-5115', 'TCGA-AK-3465']


  #slide_accuracy_test_on_target_set(target_slide_list, d_img, use_threshold=True, threshold=0.2, show_visualizer=True, show_label_accuracy=True, show_label_only=1, is_cross_model_test=is_cross_model_test, dataset_name=dataset_name)
  slide_accuracy_test_on_target_set(target_slide_list, d_img, use_threshold=True, threshold=0.2, show_visualizer=True, show_label_accuracy=True, show_label_only=None, is_cross_model_test=is_cross_model_test, dataset_name=dataset_name)

    

In [ ]:

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models
from skimage.io import imread
from skimage.transform import resize
from PIL import Image

print(model.layer4)

def show_gradcam_img(target_slide_list, model):
    model.eval()
    slide_DSS_1_cnt = 0
    
    for slide_name in target_slide_list:
        slide_dataset = CustomDataset(root = patch_samples_root, d_img=d_img, purpose='gradcam', slide_name=slide_name, show_detail=False)
        dataloader = DataLoader(dataset=slide_dataset, batch_size=1, shuffle=False, drop_last=True, pin_memory=True)
        
        if slide_DSS_1_cnt >= 8:
            break 

    
        (_, y_label) = dataloader.dataset[0]
        y_label = int(y_label[0])
        # check if the label is 1
        if y_label == 1:
            slide_DSS_1_cnt += 1
            pass
        else:
            continue

        patch_idx = 0
        true_positive_cnt_for_slide = 0
        for X, Y in dataloader:
            if true_positive_cnt_for_slide >= 3:
                print(f'end of img {slide_name}')
                break
     
            X = X.to(device)
            Y = Y.to(device)

            with torch.no_grad():
                y_pred_prob = model(X)

            # Convert the predicted probabilities to binary predictions
            y_pred_binary = torch.round(y_pred_prob)                
        
            img_name = f'{slide_name}_{patch_idx}'
            patch_idx += 1

            pred_prob = y_pred_prob[0]
            pred = y_pred_binary[0]
            pred = int(pred)
            y_answer = Y[0]
            y_answer = int(y_answer)

            # check if the prediction is positive
            if pred == 1:
                true_positive_cnt_for_slide += 1
                pass
            else:
                # 0에 대해서는 gradcam 적용 안되는 것 확인 (layer 활성화가 안되게 학습했을 것)
                continue

            pred_class_num = 0 # binary classification이라서 pred_class_num 0 밖에 못받음
            targets = [ClassifierOutputTarget(pred_class_num)]
            target_layers = [model.layer4[-1]]
            input_tensor = X
            img = X[0].permute((1,2,0))
            img = img.to('cpu')
            
            plt.title(f'{img_name}/{np.round(float(pred_prob), 2)}')
            plt.imshow(img)
            plt.show()
            # print(input_tensor.shape)
            # print(img.shape)

            # Construct the CAM object once, and then re-use it on many images:
            cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)
            # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
            grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
            #print(grayscale_cam)

            # In this example grayscale_cam has only one image in the batch:
            grayscale_cam = grayscale_cam[0, :]

            img = img.numpy()  # Convert the image tensor to a NumPy array
            visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)
            
            plt.title(f'{img_name}/{np.round(float(pred_prob), 2)}')
            plt.imshow(visualization)
            plt.show()

if show_gradcam:
    # if is_year_sort == False:
    #     img_cnt = len(os.listdir(patch_samples_root))
    #     val_set_range = range(int(img_cnt*ratio_train), int(img_cnt*(ratio_train + ratio_val)))   
    #     test_set_range = range(int(img_cnt*(ratio_train + ratio_val)), int(img_cnt))
    #     train_set_range = range(0, int(img_cnt*(ratio_train)))
    
    gradcam_target_slide_list = val_slide_list
    show_gradcam_img(gradcam_target_slide_list, model)



    # print('show example image')
    # img = np.load(f'{patch_samples_root}/TCGA-CJ-6033/TCGA-CJ-6033-1000.npy')
    # img = img / 255
    # img = img.astype(np.float32)

    # print(img.shape)
    # plt.imshow(img)
    # plt.show()
   
      

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib import cm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def output_forward_before_fc_layer(model, x):
  out = model.conv1(x)
  out = model.bn1(out)
  out = model.relu(out)
  out = model.maxpool(out)
  out = model.layer1(out)
  out = model.layer2(out)
  out = model.layer3(out)
  out = model.layer4(out)
  #print(out.shape) #torch.Size([batch_size, 512, 7, 7])
  out_2 = model.avgpool(out)
  out_2 = torch.squeeze(out_2)  
  # print(out_2.shape) # torch.Size([500, 512, 1, 1]) => torch.Size([500, 512])
  out_2 = model.fc(out_2)
  return out, out_2


class AnalyzeKernels:
  def __init__(self, kernels, patch_size):
    self.kernels = kernels
    self.patch_size = patch_size
    print(f'filter shape : {kernels.shape}')  

  def show_kernels_with_weights(self):
    # Calculate the magnitude of the weights for each kernel
    kernels = self.kernels
    weights_sum = torch.sum(torch.abs(kernels), dim=(1, 2, 3))
    sorted_tensor, indices = torch.sort(weights_sum, descending=True)
    kernels = kernels[indices]

    plt.figure(figsize=(20, 17))
    for i, kernel_img in enumerate(kernels):
      if i > 8:
        break
      kernel_img = kernel_img.cpu().detach().numpy()
      kernel_img = np.sum(kernel_img, axis=(0))
      plt.subplot(8, 8, i+1).set_title("weight sum:{0:.4f}".format(sorted_tensor[i].cpu().detach().numpy())) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
      plt.imshow(kernel_img, cmap='gray')
      plt.axis('off')


  def show_channels_of_nth_discriminative_kernel(self, nth=0):
    
    # Calculate the magnitude of the weights for each kernel
    kernels = self.kernels
    weights_sum = torch.sum(torch.abs(kernels), dim=(1, 2, 3))
    sorted_tensor, indices = torch.sort(weights_sum, descending=True)
    kernels = kernels[indices]

    target_kernel = kernels[nth]
    weights_sum = torch.sum(torch.abs(target_kernel), dim=(1, 2))
    sorted_tensor, indices = torch.sort(weights_sum, descending=True)
    target_kernel = target_kernel[indices]

    plt.figure(figsize=(20, 17))
    for i, channel in enumerate(target_kernel):
      if i > 8:
        break
      kernel_img = channel.cpu().detach().numpy()
      plt.subplot(8, 8, i+1).set_title("weight sum:{0:.4f}".format(sorted_tensor[i].cpu().detach().numpy())) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
      plt.imshow(kernel_img, cmap='gray')
      plt.axis('off')

  def get_entropy_of_filter(self, filter):
    min_val = torch.min(filter)
    filter = (filter - min_val) # 음수를 없애준다 / 전체를 위로 올리는 작업
    sum = torch.sum(filter, axis=(0,1,2))
    filter = filter / sum
    prob_sum = torch.sum(filter, axis=(0,1,2))
    #print(f'prob_sum : {prob_sum}')
    
    entropy = 0
    for i in range(filter.shape[0]):
      for j in range(filter.shape[1]):
        for k in range(filter.shape[2]):
          p = filter[i][j][k]
          if p == 0:
            continue
          #print(p)
          entropy += -p * torch.log(p)
    return entropy

  def get_entropy_of_filters(self, filters):
    
    file_name = f'filter_{PATH}.pt'
    if os.path.exists(file_name):
      entropy_list = torch.load(file_name)
    else:  
      entropy_list = torch.FloatTensor([self.get_entropy_of_filter(filter) for filter in filters])
      torch.save(entropy_list, file_name)

    print('sort by entropy complete')
    sorted_tensor, indices = torch.sort(entropy_list, descending=True)
    print(f'max entropy : {sorted_tensor[0]} min entropy : {sorted_tensor[-1]}')
    return sorted_tensor, indices

  def sort_and_show_kernels(self):
    # Calculate the magnitude of the weights for each kernel
    kernels = self.kernels
    sorted_tensor, indices = self.get_entropy_of_filters(kernels)
    self.filter_info = [sorted_tensor, indices]
    weights_sum = torch.sum(torch.abs(kernels), dim=(1, 2, 3)) # 참고 정보
    kernels = kernels[indices]

    

    plt.figure(figsize=(20, 17))
    for i, kernel_img in enumerate(kernels):
      if i > 8:
        break
      kernel_img = kernel_img.cpu().detach().numpy()
      kernel_img = np.sum(kernel_img, axis=(0)) # - value가 검은색으로 지정되어서 기준을 새로잡아주게 됨
      # print(kernel_img)
      plt.subplot(8, 8, i+1).set_title("entropy:{0:.4f}\n w:{1:.4f}".format(sorted_tensor[i].cpu().detach().numpy(), weights_sum[indices[i]].cpu().detach().numpy())) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
      plt.imshow(kernel_img, cmap='gray')
      plt.axis('off')


  def twodim_embedding_visuallize_for_filtered_images(self, num_of_filters_to_show, model, patch_size=None, dataset=None, cnt=None, is_lda=False, is_use_entropy_metric=True, use_pred_label=False):

    # filter setting
    kernels = self.kernels
    weights_sum = torch.sum(torch.abs(kernels), dim=(1, 2, 3)) # 참고 정보
    #print(weights_sum)
    
    if is_use_entropy_metric:
      filter_info = self.filter_info
      sorted_tensor = filter_info[0]
      indices = filter_info[1]
      kernels = kernels[indices]  
    else:
      sorted_tensor, indices = torch.sort(weights_sum, descending=True)
      kernels = kernels[indices]
      
    # filter setting end
    batch_size = 500
    loop_cnt = cnt // batch_size
    if cnt % batch_size != 0:
      print(f'please use multiples of {batch_size}.')
      return

    data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
    # torch.cat((tensor1, tensor2), dim=0)

    plt.figure(figsize=(20, 17))
    for nth in range(num_of_filters_to_show):
      nth_discriminative_kernel = kernels[nth]
      kernel_img = nth_discriminative_kernel.cpu().detach().numpy()
      kernel_img = np.sum(kernel_img, axis=(0))

      final_dimension_of_output = 36
      total_out_images = np.zeros((cnt, final_dimension_of_output))
      total_labels = np.zeros((cnt, 1))
      total_pred_labels = np.zeros((cnt, 1))
      i = -1
      for i, (xset, yset) in enumerate(data_loader):
        i += 1
        if i >= loop_cnt:
          break

        # cuDNN에러로 인해, continous한 메모리에 넣어줘야함
        images = xset.to(device).contiguous()
        labels = yset.to(device).contiguous()     
        #print(f'images shape : {images.shape}')
        with torch.no_grad():
          images = images.to(device)
          layer1_output, y_pred_prob = output_forward_before_fc_layer(model, images)
          pred_labels = torch.round(y_pred_prob) 
          pred_labels = pred_labels.cpu().detach().numpy()
  
        # images.cpu().detach()
        out_images = torch.nn.functional.conv2d(layer1_output, nth_discriminative_kernel.unsqueeze(0), padding=3, stride=2)
        out_images = out_images.squeeze().cpu().detach().numpy()
        
        if isinstance(labels, np.ndarray) == False:
          labels = labels.cpu().detach().numpy()
     
        length = out_images.shape[1] ** 2
        out_images = out_images.reshape(-1, length)
        total_out_images[i*(batch_size):(i+1)*(batch_size)] = out_images
        total_labels[i*(batch_size):(i+1)*(batch_size)] = labels
        total_pred_labels[i*(batch_size):(i+1)*(batch_size)] = pred_labels
        #print(out_images.shape)
      print(total_out_images.shape)

      out_images = total_out_images

      # labels = total_pred_labels
      # original_labels = total_labels
      labels = total_labels 
      pred_labels = total_pred_labels

      if is_lda:
        lda = LinearDiscriminantAnalysis()
        lda.fit(out_images, labels)
        X_transformed = lda.transform(out_images)
        print(X_transformed.shape)
      else:
        pca = PCA(n_components=2, whiten=False) 
        pca.fit(out_images)
        X_transformed = pca.fit_transform(out_images)
        print(X_transformed.shape)
      #print(weights_sum[indices[nth]])
      plt.subplot(3, 3, nth+1).set_title("entropy:{0:.4f}, weight_sum:{1:.4f}".format(
        sorted_tensor[nth].cpu().detach().numpy(), weights_sum[indices[nth]].cpu().detach().numpy()
      ))
      for i, item in enumerate(X_transformed):
        if is_lda:
          if use_pred_label:
            item = [item[0], item[0] + 2 * labels[i] + pred_labels[i]]
          else:
            item = [item[0], item[0] + labels[i]]
            
        if i > 3000:
          break
        if labels[i] == 0:
          plt.plot(item[0],item[1], 'bo') 
        elif labels[i] == 1:
          plt.plot(item[0],item[1],'ro', zorder=3) 
      plt.grid(True)
      print('kernel processed')


for i,item in enumerate(model.parameters()):
  if i == 57:
    print(f'filter shape : {item.shape}')
    kernels = item.data



if show_filter_analysis:
  is_use_entropy_metric = True
  AK = AnalyzeKernels(kernels, patch_size=patch_size)
  
  if is_use_entropy_metric:
    AK.sort_and_show_kernels()



In [ ]:
if show_filter_analysis:
  is_lda = True
  is_use_entropy_metric = True
  
  use_pred_label = True # 4줄 그어줌 (양끝 : 정답 / 중간 : 실제 레이블과 다르게 예측 (0을 1로, 1을 0으로))
  cnt = 20000
  AK.twodim_embedding_visuallize_for_filtered_images(num_of_filters_to_show=9, model=model, patch_size=patch_size, dataset=val_dataset, cnt=cnt, is_lda=is_lda, is_use_entropy_metric=is_use_entropy_metric, use_pred_label=use_pred_label)
  AK.twodim_embedding_visuallize_for_filtered_images(num_of_filters_to_show=9, model=model, patch_size=patch_size, dataset=test_dataset, cnt=cnt, is_lda=is_lda, is_use_entropy_metric=is_use_entropy_metric, use_pred_label=use_pred_label)
  AK.twodim_embedding_visuallize_for_filtered_images(num_of_filters_to_show=9, model=model, patch_size=patch_size, dataset=dataset, cnt=cnt, is_lda=is_lda, is_use_entropy_metric=is_use_entropy_metric, use_pred_label=use_pred_label)
